# bai toan cho tap du lieu csv: 
D:/Python/2024/test_hadoop/hoc_spark/Orders.csv
- tao cot moi ten la total_price la gia tri cua product price * quantity
- sua gia tri trong product price: 
    - neu product name san pham la Canon thi tang gia san pham len 20%
    - neu la Xerox thi tang gia len 50%
    - cac san pham khac giu nguyen gia

In [119]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [17]:
ss = SparkSession.builder.appName("dataframe 3").getOrCreate()

In [24]:
schema_struct = StructType([
    StructField("catelogy", StringType()),
    StructField("name", StringType()),
    StructField("price", FloatType()),
    StructField("quantity", IntegerType())
])

In [36]:
df = ss.read \
    .format("csv") \
    .option("header", "False") \
    .schema(schema_struct) \
    .load("D:/Python/2024/test_hadoop/hoc_spark/data/products.csv")

In [41]:
df.printSchema()

root
 |-- catelogy: string (nullable = true)
 |-- name: string (nullable = true)
 |-- price: float (nullable = true)
 |-- quantity: integer (nullable = true)



In [43]:
df = df.withColumn("total_price", col("price") * col("quantity"))

In [75]:
df.select("*", expr("price * quantity as total_demo")).show()

+---------------+--------------------+------+--------+-----------+----------+
|       catelogy|                name| price|quantity|total_price|total_demo|
+---------------+--------------------+------+--------+-----------+----------+
|Office Supplies|            Avery 49|  2.88|       2|       5.76|      5.76|
|Office Supplies|SANFORD Liquid Ac...|  2.84|       4|      11.36|     11.36|
|Office Supplies|          Xerox 1968|  6.68|       7|      46.76|     46.76|
|Office Supplies|Acme� Preferred S...|  5.68|       7|      39.76|     39.76|
|     Technology|                 V70|205.99|       8|    1647.92|   1647.92|
|Office Supplies|           Xerox 194| 55.48|      37|    2052.76|   2052.76|
|     Technology|Canon S750 Color ...|120.97|      12|    1451.64|   1451.64|
|      Furniture|Global Troy� Exec...|500.98|      12|  6011.7603| 6011.7603|
|Office Supplies|          Xerox 1930|  6.48|      18|     116.64|    116.64|
|Office Supplies|Kensington 6 Outl...| 90.24|      16|    1443.8

In [77]:
df.show(5, False)

+---------------+----------------------------------------------+------+--------+-----------+
|catelogy       |name                                          |price |quantity|total_price|
+---------------+----------------------------------------------+------+--------+-----------+
|Office Supplies|Avery 49                                      |2.88  |2       |5.76       |
|Office Supplies|SANFORD Liquid Accent� Tank-Style Highlighters|2.84  |4       |11.36      |
|Office Supplies|Xerox 1968                                    |6.68  |7       |46.76      |
|Office Supplies|Acme� Preferred Stainless Steel Scissors      |5.68  |7       |39.76      |
|Technology     |V70                                           |205.99|8       |1647.92    |
+---------------+----------------------------------------------+------+--------+-----------+
only showing top 5 rows



In [114]:
df.withColumn("price_new", 
              when(col("name").contains("Canon"), col("price") * 1.2)
              .when(col("name").contains("Xerox"), col("price") * 1.5)
              .otherwise(col("price"))).show()

+---------------+--------------------+------+--------+-----------+------------------+
|       catelogy|                name| price|quantity|total_price|         price_new|
+---------------+--------------------+------+--------+-----------+------------------+
|Office Supplies|            Avery 49|  2.88|       2|       5.76| 2.880000114440918|
|Office Supplies|SANFORD Liquid Ac...|  2.84|       4|      11.36|2.8399999141693115|
|Office Supplies|          Xerox 1968|  6.68|       7|      46.76|10.019999742507935|
|Office Supplies|Acme� Preferred S...|  5.68|       7|      39.76| 5.679999828338623|
|     Technology|                 V70|205.99|       8|    1647.92|205.99000549316406|
|Office Supplies|           Xerox 194| 55.48|      37|    2052.76| 83.21999931335449|
|     Technology|Canon S750 Color ...|120.97|      12|    1451.64|145.16400146484375|
|      Furniture|Global Troy� Exec...|500.98|      12|  6011.7603| 500.9800109863281|
|Office Supplies|          Xerox 1930|  6.48|      18|

In [121]:
df.withColumn("price_new", 
              when(col("name").contains("Canon"), hii(col("price") * 1.2, 2))
              .when(col("name").contains("Xerox"), hii(col("price") * 1.5, 2))
              .otherwise(hii(col("price"), 2))).show()

+---------------+--------------------+------+--------+-----------+------------------+
|       catelogy|                name| price|quantity|total_price|         price_new|
+---------------+--------------------+------+--------+-----------+------------------+
|Office Supplies|            Avery 49|  2.88|       2|       5.76| 2.880000114440918|
|Office Supplies|SANFORD Liquid Ac...|  2.84|       4|      11.36|2.8399999141693115|
|Office Supplies|          Xerox 1968|  6.68|       7|      46.76|             10.02|
|Office Supplies|Acme� Preferred S...|  5.68|       7|      39.76| 5.679999828338623|
|     Technology|                 V70|205.99|       8|    1647.92|205.99000549316406|
|Office Supplies|           Xerox 194| 55.48|      37|    2052.76|             83.22|
|     Technology|Canon S750 Color ...|120.97|      12|    1451.64|            145.16|
|      Furniture|Global Troy� Exec...|500.98|      12|  6011.7603| 500.9800109863281|
|Office Supplies|          Xerox 1930|  6.48|      18|